In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm.notebook import tqdm

In [2]:
# Input and output directories
#MOT17 video = 'MOT17-02-DPM'
#UrbanTracker video= 'rene_video'
tracking_outputs_dir = r"C:\Users\User\Documents\vision\UrbanTracker\rouen_video"
#image_folder = r"C:\Users\User\Documents\MOT17\MOT17\train\MOT17-02-DPM\img1"  # Folder with extracted images
output_dir = r"C:\Users\User\Documents\vision\UrbanTracker\rouen_video\analysis_results"

video_path = r"C:\Users\User\Documents\vision\UrbanTracker\rouen_video\rouen_video.avi"  # Path to your video file
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Load gt
gt_df = pd.read_csv(r"C:\Users\User\Documents\vision\UrbanTracker\rouen_video\gt.txt")
gt_df = gt_df.iloc[:, 0:6]
gt_df.columns = ['frame', 'gt_id', 'x', 'y', 'w', 'h']
gt_df[['frame', 'x', 'y', 'w', 'h']] = gt_df[['frame', 'x', 'y', 'w', 'h']].apply(pd.to_numeric, errors='coerce')

In [4]:
iou_threshold = 0.4

# IoU calculation function
def compute_iou(boxA, boxB):
    # Convert (x, y, w, h) => (x1, y1, x2, y2)
    xA1, yA1, wA, hA = boxA
    xB1, yB1, wB, hB = boxB
    xA2, yA2 = xA1 + wA, yA1 + hA
    xB2, yB2 = xB1 + wB, yB1 + hB

    # Compute intersection rectangle
    inter_x1 = max(xA1, xB1)
    inter_y1 = max(yA1, yB1)
    inter_x2 = min(xA2, xB2)
    inter_y2 = min(yA2, yB2)
    inter_w = max(0, inter_x2 - inter_x1)
    inter_h = max(0, inter_y2 - inter_y1)
    inter_area = inter_w * inter_h

    # Compute union area
    areaA = wA * hA
    areaB = wB * hB
    union_area = areaA + areaB - inter_area

    if union_area == 0:
        return 0
    return inter_area / union_area

In [5]:
def get_top_bad_frames_by_threshold(matches_df, top_n=3):

    # Count how many bad matches are in each frame
    frame_error_counts = matches_df.groupby('frame').size().sort_values(ascending=False)

    # Select top N worst frames by number of bad matches
    top_bad_frame_ids = frame_error_counts.head(top_n).index.tolist()

    # Return the full match data for these frames
    return matches_df[matches_df['frame'].isin(top_bad_frame_ids)].sort_values(by=['frame', 'iou'])

In [6]:
# Function to draw bounding boxes
def draw_boxes(image, boxes, color, label_prefix):
    for box in boxes:
        x, y, w, h = int(box['x']), int(box['y']), int(box['w']), int(box['h'])
        obj_id = int(box['id'])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, f"{label_prefix} {obj_id}", (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image

In [7]:
# Find all relevant CSV files that match the YOLO tracking output pattern
csv_files = [f for f in os.listdir(tracking_outputs_dir) if f.startswith("yolo_tracking_output") and f.endswith(".csv")]

# Loop through each file and process it
for file in tqdm(csv_files):
    model_name = file.replace("yolo_tracking_output_", "").replace(".csv", "")
    file_path = os.path.join(tracking_outputs_dir, file)

    # Load the CSV file into a DataFrame
    yolo_df = pd.read_csv(file_path,header=0)
    yolo_df.columns = ['frame', 'yolo_id', 'x', 'y', 'w', 'h']
    yolo_df[['frame', 'x', 'y', 'w', 'h']] = yolo_df[['frame', 'x', 'y', 'w', 'h']].apply(pd.to_numeric, errors='coerce')

    # Match YOLO predictions to GT by frame and IoU
    matches = []
    
    # Go frame by frame
    for frame in sorted(yolo_df['frame'].unique()):
        yolo_frame = yolo_df[yolo_df['frame'] == frame]
        gt_frame = gt_df[gt_df['frame'] == frame]
    
        # For each YOLO detection in the frame
        for _, yolo_row in yolo_frame.iterrows():
            best_iou = 0
            best_gt_id = None
            yolo_box = [yolo_row['x'], yolo_row['y'], yolo_row['w'], yolo_row['h']]
    
            # Compare to each GT box in the same frame
            for _, gt_row in gt_frame.iterrows():
                gt_box = [gt_row['x'], gt_row['y'], gt_row['w'], gt_row['h']]
                iou = compute_iou(yolo_box, gt_box)
    
                if iou > best_iou:
                    best_iou = iou
                    best_gt_id = gt_row['gt_id']
    
            # Save if IoU is above threshold
            if best_iou >= iou_threshold:
                matches.append({
                    'frame': frame,
                    'yolo_id': yolo_row['yolo_id'],
                    'gt_id': best_gt_id,
                    'iou': best_iou
                })
    
    # Convert to DataFrame and save or display
    matches_df = pd.DataFrame(matches)

    # Get the top n worst frames based on the chosen threshold
    top_bad_frames_custom = get_top_bad_frames_by_threshold(matches_df, top_n=3)
    top_bad_frames_custom.to_csv(os.path.join(output_dir, f"{model_name}_top_bad_frames_custom.csv"), index=False)
    worst_frame_ids = top_bad_frames_custom['frame'].unique()
    
    
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"Failed to open video: {video_path}")
    
    # Loop over each bad frame
    for frame_id in worst_frame_ids:
        # Set video to the specific frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, img = cap.read()
        if not ret:
            #print(f"Frame not found: {frame_id}")
            continue
    
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        # Get GT and YOLO boxes for current frame
        gt_boxes = gt_df[gt_df['frame'] == frame_id][['x', 'y', 'w', 'h', 'gt_id']].rename(columns={'gt_id': 'id'})
        yolo_boxes = yolo_df[yolo_df['frame'] == frame_id][['x', 'y', 'w', 'h', 'yolo_id']].rename(columns={'yolo_id': 'id'})
    
        # 1. GT only
        gt_img = draw_boxes(img_rgb.copy(), gt_boxes.to_dict('records'), (0, 255, 0), 'GT')
        plt.imshow(gt_img)
        plt.title(f"Frame {frame_id} – Ground Truth")
        plt.axis('off')
        plt.savefig(os.path.join(output_dir, f"{model_name}Frame{frame_id}_GroundTruth.png"))
        plt.close()
    
        # 2. YOLO only
        yolo_img = draw_boxes(img_rgb.copy(), yolo_boxes.to_dict('records'), (255, 0, 0), 'YOLO')
        plt.imshow(yolo_img)
        plt.title(f"Frame {frame_id} – YOLO Prediction")
        plt.axis('off')
        plt.savefig(os.path.join(output_dir, f"{model_name}Frame{frame_id}_YOLO.png"))
        plt.close()
    
        # 3. Combined
        combined_img = draw_boxes(img_rgb.copy(), gt_boxes.to_dict('records'), (0, 255, 0), 'GT')
        combined_img = draw_boxes(combined_img, yolo_boxes.to_dict('records'), (255, 0, 0), 'YOLO')
        plt.imshow(combined_img)
        plt.title(f"Frame {frame_id} – GT (green) vs YOLO (red)")
        plt.axis('off')
        plt.savefig(os.path.join(output_dir, f"{model_name}Frame{frame_id}_Combined.png"))
        plt.close()
    
    cap.release()



  0%|          | 0/12 [00:00<?, ?it/s]